In [1]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import os
os.chdir("c:\\Users\\swart\\Desktop\\secure-mpc-main\\basic_networking")
from client import Client
from server import Server
os.chdir("c:\\Users\\swart\\Desktop\\secure-mpc-main\\ecgPreprocessedData")
from Data.SignalDataset import SignalDataset
from PickleFileUtils import read_in_pickle_file

In [2]:
complete_data_set = read_in_pickle_file("data\\CompleteDataSetFeatures.pickle")
original_training_data_set = read_in_pickle_file("data\\OriginalTrainingDataSetFeatures.pickle")
original_validation_data_set = read_in_pickle_file("data\\OriginalValidationDataSetFeatures.pickle")

print(len(complete_data_set))
print(len(original_training_data_set))
print(len(original_validation_data_set))

6029
5834
195


In [3]:
# get labels
labels = complete_data_set.labels

# get data
data = complete_data_set.data

data,labels = shuffle(data, labels)
X_class = np.array(data)
y_class = np.array(labels)

In [4]:
alice = Client()
server = Server()

def test_multiple_points(X_train, X_test, y_train, y_test, num_neighbors, client: Client, server: Server, rand_range):
	server.set_features_labels(X_train, y_train)
	y_pred = []
	for test_row in X_test:
		prediction = client.predict_classification(test_row, num_neighbors, server, rand_range)
		y_pred.append(prediction)

	f1_binary = f1_score(y_test, y_pred, average="binary")
	accuracy = accuracy_score(y_test, y_pred)

	# print("ypred:", y_pred)
	# print("ytest:", y_test)

	return f1_binary, accuracy



In [5]:
#testing

def test(rand_range):

    numFolds = 10
    num_neighbors = 9
    stratifiedKFold = StratifiedKFold(
        n_splits=numFolds, shuffle=True, random_state=86
    )

    count = 1
    avgF1 = 0
    avgAcc = 0

    X = X_class
    y = y_class

    server = Server()
    alice = Client()
    rand_seed = 4
    rand_range = 100

    for train_index, test_index in stratifiedKFold.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        f1, acc = test_multiple_points(X_train, X_test, y_train, y_test, num_neighbors, alice, server, rand_range)
        avgF1 += f1
        avgAcc += acc

        #print(f"\tFold {count}:: Accuracy: {acc}, F1 score: {f1}")

        count += 1

    avgF1 = avgF1/numFolds
    avgAcc = avgAcc/numFolds

    return avgAcc, avgF1



In [6]:
# test with multiple lagrange max x-points

x_max_list = [100,1000,10000,100000,1000000]

print("Testing 2017 AF Dataset - SMPC")

for x_max in x_max_list:
    avgAcc, avgF1 = test(x_max)
    print(f"Average accuracy: {avgAcc}, Average F1 score: {avgF1}, Lagrange max x-coord: {x_max}")

Testing 2017 AF Dataset - SMPC
S(0):-7.450580596923828e-09	(30173478.000000004, 47756358.0, 6154008.0)
S(0):-1.4901161193847656e-08	(76231325.00000001, 120653324.99999999, 15547700.0)
S(0):-7.450580596923828e-09	(59486861.0, 94151420.99999999, 12132596.0)
S(0):-3.725290298461914e-09	(38968550.0, 61676549.99999999, 7947800.0)
S(0):-1.862645149230957e-09	(22498932.0, 35609651.99999999, 4588752.0)
S(0):-1.4901161193847656e-08	(47689625.0, 75479624.99999999, 9726500.0)
S(0):-1.862645149230957e-08	(15301042.000000002, 24217361.999999993, 3120712.0)
S(0):-1.862645149230957e-09	(24464177.000000004, 38720097.0, 4989572.0)
S(0):-7.450580596923828e-09	(30001459.000000004, 47484099.0, 6118924.0)
S(0):-2.2351741790771484e-08	(60630451.0, 95961410.99999999, 12365836.0)
S(0):-3.725290298461914e-09	(34319232.0, 54317951.99999999, 6999552.000000001)
S(0):-2.9802322387695312e-08	(53659357.0, 84928076.99999999, 10944052.0)
S(0):-3.725290298461914e-09	(43327646.0, 68575805.99999999, 8836856.0)
S(0):-7.45